#Preprocessing

## Gestione delle dipendenze

Import Pandas per il preprocessing. Eventualmente utilizzabile SKLearn (scikit-learn).

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder

## Import dei contenuti in Py (pandas.DataFrame)

In [2]:
CSV = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/housing_dirty.csv"

df = pd.read_csv(CSV, index_col=0)

df_columns = df.columns

In [3]:
df.shape

(506, 14)

In [4]:
for column in df.columns:
    if(df[column].dtype == "object"):
        print(df[column].value_counts())
    

CRIM
HIGH         130
LOW          127
VERY HIGH    127
MODERATE     122
Name: count, dtype: int64
CHAS
NO     471
YES     35
Name: count, dtype: int64


In [5]:
print(df.isna().sum())

CRIM         0
ZN           2
INDUS        3
CHAS         0
NOX          7
RM           5
AGE          4
DIS          5
RAD          3
TAX          2
PTRATIO      5
B            3
LSTAT      199
PRICE        4
dtype: int64


In [6]:
#remove columns with isna().sum() > 30%

df = df.dropna(axis=1, thresh=df.shape[0]*0.7) #Almeno il 70% dei valori delle colonne deve essere non nulli

df = df.dropna(thresh=df.shape[1]*0.75) #Almeno il 75% dei valori delle righe deve essere non nulli
 
df = df.dropna(subset=["PRICE"]) #Rimuoviamo le righe con valori nulli nella colonna PRICE


print(df.isna().sum())

CRIM       0
ZN         0
INDUS      1
CHAS       0
NOX        4
RM         0
AGE        0
DIS        1
RAD        0
TAX        0
PTRATIO    3
B          1
PRICE      0
dtype: int64


In [7]:
for column in df.columns: # Verifica se ci sono colonne con valori nulli e sostituisci i valori nulli con la moda o la media
    if(column == "PRICE"):
        continue
    if df[column].dtype == "object": #verifica tipo di dato
        replace_with = df[column].mode()[0]
        df[column] = df[column].fillna(replace_with)
    else:
        replace_with = round(df[column].mean(), 1) #arrotondamento scelto in base ad altri valori del dataset
        df[column] = df[column].fillna(replace_with)

print(df.isna().sum())


CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
PRICE      0
dtype: int64


In [8]:
#Column CRIM -> ordering #LabelEncoding

mapper = {"LOW":0, "MODERATE":1, "HIGH":2, "VERY HIGH":3}

fmap = np.vectorize(lambda x: mapper[x])

df["CRIM"] = fmap(df["CRIM"])

In [9]:
#trasformazione con ColumnTransformer:

from sklearn.compose import ColumnTransformer #per applicare le trasformazioni alle colonne
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler #per normalizzare i dati
from sklearn.preprocessing import LabelEncoder #per trasformare le variabili categoriche in numeriche

column_transformer = ColumnTransformer(
    [
        ("onehot", OneHotEncoder(), ["CHAS"]), #da non standardizzare
        ("scaler", StandardScaler(), ["CRIM", "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B"]) #non applicare sulla variabile da prevedere, da applicare solo sulle variabili continue e categoriche ordinali
    ],
    remainder="passthrough"
)

df_test = column_transformer.fit_transform(df)

df.shape


(497, 13)

In [10]:
#smote -> frequenza rispetto alla variabile da prevedere (rischioso, come ultima spiaggia)

#dare il giusto peso agli outliers

In [11]:
# Determina il numero di colonne generate

onehot_columns = column_transformer.named_transformers_['onehot'].get_feature_names_out(["CHAS"])
scaler_columns = ["CRIM", "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B"]
all_columns = list(onehot_columns) + scaler_columns + ["PRICE"] 

print(all_columns)

['CHAS_NO', 'CHAS_YES', 'CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'PRICE']


In [12]:
df_clean = pd.DataFrame(df_test, columns=all_columns)

df_clean.head(15)

,CHAS_NO,CHAS_YES,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,PRICE
0,1.0,0.0,-1.333590,0.276292,-1.287904,0.334164,0.471869,-0.114472,-0.297837,-0.984211,-0.669687,-1.476389,0.439288,24.0
1,1.0,0.0,-1.333590,-0.491081,-0.594910,0.025586,0.395235,0.370547,-0.297150,-0.870030,-0.988387,-0.311756,0.439288,21.6
2,1.0,0.0,-1.333590,-0.491081,-0.594910,0.025586,0.775421,-0.259623,-0.297150,-0.870030,-0.988387,-0.311756,0.394882,34.7
3,1.0,0.0,-1.333590,-0.491081,-1.306831,-0.023608,0.682365,-0.801287,-0.296293,-0.755849,-1.106424,0.107512,0.414521,33.4
4,1.0,0.0,-1.333590,-0.491081,-1.306831,-0.023608,0.756511,-0.503903,-0.296293,-0.755849,-1.106424,0.107512,0.439288,36.2
5,1.0,0.0,-1.333590,-0.491081,-1.306831,-0.023608,-2.796821,-0.344590,-0.296293,-0.755849,-1.106424,0.107512,0.408957,28.7
6,1.0,0.0,-0.442738,0.041817,-0.478440,0.271554,0.191706,-0.064908,-0.296686,-0.527486,-0.581160,-1.522975,0.425104,22.9
7,1.0,0.0,-0.442738,0.041817,-0.478440,0.271554,0.271326,0.979476,-0.296380,-0.527486,-0.581160,-1.522975,0.439288,27.1
8,1.0,0.0,-0.442738,0.041817,-0.478440,0.271554,0.002111,1.117547,-0.296277,-0.527486,-0.581160,-1.522975,0.327238,16.5
9,1.0,0.0,-0.442738,0.041817,-0.478440,0.271554,0.187725,0.618367,-0.295878,-0.527486,-0.581160,-1.522975,0.328110,18.9


In [13]:
#Salvataggio del dataset pulito

path = "housing_clean.csv"
df_clean.to_csv(path, index=False) #salva il csv senza gli indici di riga

In [14]:
#calcolo della correlazione - dopo la categorizzazione

correlation = df_clean.drop(onehot_columns, axis=1).corr()

correlation["PRICE"].sort_values(ascending=False)

correlation["PRICE"]

CRIM      -0.398107
ZN         0.366383
INDUS     -0.487688
NOX       -0.105391
RM         0.157762
AGE       -0.378943
DIS        0.100397
RAD       -0.386924
TAX       -0.471036
PTRATIO   -0.505041
B          0.335481
PRICE      1.000000
Name: PRICE, dtype: float64

In [15]:
#selezione delle variabili correlate

df_clean = df_clean.drop(["NOX", "RM", "DIS"], axis=1)

df_clean.head()

,CHAS_NO,CHAS_YES,CRIM,ZN,INDUS,AGE,RAD,TAX,PTRATIO,B,PRICE
0,1.0,0.0,-1.33359,0.276292,-1.287904,-0.114472,-0.984211,-0.669687,-1.476389,0.439288,24.0
1,1.0,0.0,-1.33359,-0.491081,-0.594910,0.370547,-0.870030,-0.988387,-0.311756,0.439288,21.6
2,1.0,0.0,-1.33359,-0.491081,-0.594910,-0.259623,-0.870030,-0.988387,-0.311756,0.394882,34.7
3,1.0,0.0,-1.33359,-0.491081,-1.306831,-0.801287,-0.755849,-1.106424,0.107512,0.414521,33.4
4,1.0,0.0,-1.33359,-0.491081,-1.306831,-0.503903,-0.755849,-1.106424,0.107512,0.439288,36.2


In [16]:
#split del dataset

from sklearn.model_selection import train_test_split

X = df_clean.drop("PRICE", axis=1)
y = df_clean["PRICE"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
#tranining del modello di regressione lineare

from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train) #train

LinearRegression()

In [18]:
#verifica delle metriche del modello r2 e mse

from sklearn.metrics import r2_score, mean_squared_error

y_pred = model.predict(X_test) #inference

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

In [19]:
r2, mse

(0.43133007150367186, 43.90982805672655)